<a href="https://colab.research.google.com/github/elangbijak4/Quantum_Algorithm_Almudaya/blob/main/Rev3_Quantum_PoW_PoS_Blockchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cirq

In [2]:
import hashlib
import datetime as date
import cirq
import random

In [5]:
class Block:
    def __init__(self, index, timestamp, data, previous_hash, nonce=0, validator=None):
        self.index = index
        self.timestamp = timestamp
        self.data = data
        self.previous_hash = previous_hash
        self.nonce = nonce
        self.validator = validator
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        sha = hashlib.sha256()
        sha.update(str(self.index).encode('utf-8') +
                   str(self.timestamp).encode('utf-8') +
                   str(self.data).encode('utf-8') +
                   str(self.previous_hash).encode('utf-8') +
                   str(self.nonce).encode('utf-8') +
                   str(self.validator).encode('utf-8'))
        return sha.hexdigest()

class QuantumProofOfWork:
    def __init__(self, difficulty):
        self.difficulty = difficulty

    def run_quantum_circuit(self, nonce):
        qubits = cirq.LineQubit.range(3)

        # Contoh sirkuit kuantum sederhana untuk PoW
        circuit = cirq.Circuit(
            cirq.H(qubits[0]),
            cirq.CX(qubits[0], qubits[1]),
            cirq.X(qubits[1]) ** nonce,
            cirq.H(qubits[1]),
            cirq.measure(qubits[1], key='result')
        )

        simulator = cirq.Simulator()
        result = simulator.run(circuit, repetitions=1)

        measured_result = result.measurements['result'][0][0]
        return int(measured_result)

    def mine(self, block):
        nonce = 0
        target = 2 ** (256 - self.difficulty)
        while True:
            block.nonce = nonce
            hash_result = int(block.calculate_hash(), 16)
            quantum_hash = self.run_quantum_circuit(nonce)
            if hash_result < target and quantum_hash == 0:
                return nonce, quantum_hash
            else:
                nonce += 1

class QuantumProofOfStake:
    def __init__(self, stakeholders):
        self.stakeholders = stakeholders

    def quantum_random_selection(self):
        # Buat sirkuit kuantum untuk memilih validator berdasarkan stake
        qubits = cirq.LineQubit.range(len(self.stakeholders))

        circuit = cirq.Circuit(
            cirq.H.on_each(*qubits),  # Hadamard gate untuk menciptakan superposisi
            cirq.measure(*qubits, key='result')  # Ukur hasil superposisi
        )

        simulator = cirq.Simulator()
        result = simulator.run(circuit, repetitions=1)

        # Hitung probabilitas masing-masing pemilik stake dipilih
        prob_distribution = {}
        for i, stakeholder in enumerate(self.stakeholders):
            measurement_result = result.measurements['result'][0][i]
            if measurement_result == 1:
                prob_distribution[stakeholder] = True
            else:
                prob_distribution[stakeholder] = False

        return prob_distribution

    def validate_stake(self, block):
        # Validasi stake: Blok harus memiliki indeks yang merupakan kelipatan dari nilai stake
        # The original code was checking if the index was a multiple of the number of stakeholders.
        # This change checks if the index is a multiple of any individual stakeholder's stake (assumed to be 1 for simplicity here).
        return block.index % 1 == 0

class QuantumByzantineFaultTolerance:
    def __init__(self, threshold):
        self.threshold = threshold

    def reach_consensus(self, blocks):
        # Contoh algoritma Byzantine Fault Tolerance kuantum
        return len(blocks) > self.threshold

class Blockchain:
    def __init__(self, consensus_type, difficulty_or_stake_or_threshold, stakeholders=None):
        self.chain = [self.create_genesis_block()]
        self.consensus_type = consensus_type

        if self.consensus_type == 'qpow':
            self.qproof = QuantumProofOfWork(difficulty_or_stake_or_threshold)
        elif self.consensus_type == 'qpos':
            self.qproof = QuantumProofOfStake(stakeholders)
        elif self.consensus_type == 'qbft':
            self.qproof = QuantumByzantineFaultTolerance(difficulty_or_stake_or_threshold)

    def create_genesis_block(self):
        return Block(0, date.datetime.now(), "Genesis Block", "0")

    def get_latest_block(self):
        return self.chain[-1]

    def add_block(self, new_block):
        new_block.previous_hash = self.get_latest_block().hash

        if self.consensus_type == 'qpow':
            nonce, quantum_hash = self.qproof.mine(new_block)
            new_block.nonce = nonce
        elif self.consensus_type == 'qpos':
            if not self.qproof.validate_stake(new_block):
                raise ValueError("Invalid stake for this block.")

            # Pilih validator menggunakan sirkuit kuantum
            prob_distribution = self.qproof.quantum_random_selection()
            selected_validator = random.choices(list(prob_distribution.keys()), weights=list(prob_distribution.values()))[0]
            new_block.validator = selected_validator

        elif self.consensus_type == 'qbft':
            # QBFT tidak memerlukan validasi tambahan di sini
            pass

        new_block.hash = new_block.calculate_hash()
        self.chain.append(new_block)
        print(f"Block mined with Nonce: {new_block.nonce if self.consensus_type == 'qpow' else 'N/A'} and Quantum Hash: {quantum_hash if self.consensus_type == 'qpow' else 'N/A'}, Validator: {new_block.validator if self.consensus_type == 'qpos' else 'N/A'}")

# Contoh penggunaan

# Membuat blockchain dengan Quantum PoW
difficulty = 4
my_blockchain_qpow = Blockchain('qpow', difficulty)

# Menambahkan beberapa blok menggunakan Quantum PoW
my_blockchain_qpow.add_block(Block(1, date.datetime.now(), {'amount': 4}, ''))
my_blockchain_qpow.add_block(Block(2, date.datetime.now(), {'amount': 10}, ''))

# Membuat blockchain dengan Quantum PoS
stakeholders = ['Alice', 'Bob', 'Charlie', 'David']
my_blockchain_qpos = Blockchain('qpos', None, stakeholders)

# Menambahkan beberapa blok menggunakan Quantum PoS
# Disesuaikan agar blok pertama memiliki indeks yang valid berdasarkan stake
my_blockchain_qpos.add_block(Block(2, date.datetime.now(), {'amount': 4}, ''))
my_blockchain_qpos.add_block(Block(4, date.datetime.now(), {'amount': 10}, ''))

# Membuat blockchain dengan Quantum Byzantine Fault Tolerance (QBFT)
threshold = 1
my_blockchain_qbft = Blockchain('qbft', threshold)

# Menambahkan beberapa blok menggunakan Quantum Byzantine Fault Tolerance (QBFT)
my_blockchain_qbft.add_block(Block(1, date.datetime.now(), {'amount': 4}, ''))
my_blockchain_qbft.add_block(Block(2, date.datetime.now(), {'amount': 10}, ''))

# Menampilkan blockchain
def print_blockchain(blockchain):
    for block in blockchain.chain:
        print(f"Index: {block.index}")
        print(f"Timestamp: {block.timestamp}")
        print(f"Data: {block.data}")
        print(f"Previous Hash: {block.previous_hash}")
        print(f"Hash: {block.hash}")
        print(f"Nonce: {block.nonce if blockchain.consensus_type == 'qpow' else 'N/A'}")
        print(f"Validator: {block.validator if blockchain.consensus_type == 'qpos' else 'N/A'}")
        print("\n")

print("Blockchain with Quantum Proof of Work (QPoW):")
print_blockchain(my_blockchain_qpow)

print("Blockchain with Quantum Proof of Stake (QPoS):")
print_blockchain(my_blockchain_qpos)

print("Blockchain with Quantum Byzantine Fault Tolerance (QBFT):")
print_blockchain(my_blockchain_qbft)

Block mined with Nonce: 27 and Quantum Hash: 0, Validator: N/A
Block mined with Nonce: 0 and Quantum Hash: 0, Validator: N/A
Block mined with Nonce: N/A and Quantum Hash: N/A, Validator: Alice
Block mined with Nonce: N/A and Quantum Hash: N/A, Validator: Charlie
Block mined with Nonce: N/A and Quantum Hash: N/A, Validator: N/A
Block mined with Nonce: N/A and Quantum Hash: N/A, Validator: N/A
Blockchain with Quantum Proof of Work (QPoW):
Index: 0
Timestamp: 2024-06-18 17:21:54.238190
Data: Genesis Block
Previous Hash: 0
Hash: a7bcdd3646fdb7e93affdedf23f007a01d5df6954240b14282e8f8d0900eb583
Nonce: 0
Validator: N/A


Index: 1
Timestamp: 2024-06-18 17:21:54.238344
Data: {'amount': 4}
Previous Hash: a7bcdd3646fdb7e93affdedf23f007a01d5df6954240b14282e8f8d0900eb583
Hash: 03c29b593519da127a0fedba5667696f48546432e06838349f4d8e2a6e5b1d11
Nonce: 27
Validator: N/A


Index: 2
Timestamp: 2024-06-18 17:21:54.313716
Data: {'amount': 10}
Previous Hash: 03c29b593519da127a0fedba5667696f48546432e06838349f